# Introduction
Data preparation of the TechDebt dataset. Concretely, from the following tables:
- GIT_COMMITS
- GIT_COMMITS_CHANGES
- JIRA_ISSUES
- SONAR_ANALYSIS
- SONAR_ISSUES
- SONAR_MEASURES

## Library Packages

In [139]:
# Import libraries and packages
# Miscellaneous libraries
import numpy as np
import pandas as pd
import os
from datetime import datetime
import collections
from prettytable import PrettyTable

## Data Preparation

#### Define the path of the files

In [140]:
# Define the path of the data files
path = '../data/raw/'
path_git_commits = path + 'GIT_COMMITS.csv'
path_git_commits_changes = path + 'GIT_COMMITS_CHANGES.csv'
path_jira_issues = path + 'JIRA_ISSUES.csv'
path_sonar_analysis = path + 'SONAR_ANALYSIS.csv'
path_sonar_issues = path + 'SONAR_ISSUES.csv'
path_sonar_measures = path + 'SONAR_MEASURES.csv'

# Ensure the input file exist
assert os.path.isfile(path_git_commits), f'{path_git_commits} not found. Is it a file?'
assert os.path.isfile(path_git_commits_changes), f'{path_git_commits_changes} not found. Is it a file?'
assert os.path.isfile(path_jira_issues), f'{path_jira_issues} not found. Is it a file?'
assert os.path.isfile(path_sonar_analysis), f'{path_sonar_analysis} not found. Is it a file?'
assert os.path.isfile(path_sonar_issues), f'{path_sonar_issues} not found. Is it a file?'
assert os.path.isfile(path_sonar_measures), f'{path_sonar_measures} not found. Is it a file?'

#### Read the files

In [141]:
# Read the files
#git_commits_changes = pd.read_csv(path_git_commits_changes)
#git_commits = pd.read_csv(path_git_commits)
jira_issues = pd.read_csv(path_jira_issues)
sonar_analysis = pd.read_csv(path_sonar_analysis)
sonar_issues = pd.read_csv(path_sonar_issues)
sonar_measures = pd.read_csv(path_sonar_measures)

#### Define selected variables
In the following section we are only selecting the useful variables for the project. The election process has been studied previusly, in the Data Understanding step.

In [142]:
# Define variables of interest for each dataframe
#git_commits_changes_names = ['COMMIT_HASH','DATE','LINES_ADDED','LINES_REMOVED']
#git_commits_names = ['PROJECT_ID','COMMIT_HASH','AUTHOR','AUTHOR_DATE','AUTHOR_TIMEZONE']
jira_issues_names = ['HASH']
sonar_analysis_names = ['PROJECT_ID','ANALYSIS_KEY','REVISION']
sonar_issues_names = ['CREATION_ANALYSIS_KEY','SEVERITY','STATUS','EFFORT','MESSAGE','START_LINE','END_LINE','CLOSE_ANALYSIS_KEY']
sonar_measures_names = ['analysis_key','complexity' ,'cognitive_complexity', 'coverage', 'duplicated_blocks', 'duplicated_files', 
                        'duplicated_lines_density', 'violations','blocker_violations','critical_violations','major_violations','minor_violations','info_violations','false_positive_issues','open_issues','reopened_issues','confirmed_issues', 'sqale_debt_ratio','code_smells','bugs','reliability_rating','vulnerabilities','security_rating','files', 'comment_lines_density']

In [143]:
# Select variables of interest
#git_commits = git_commits[git_commits_names]
jira_issues = jira_issues[jira_issues_names]
sonar_analysis = sonar_analysis[sonar_analysis_names]
sonar_issues = sonar_issues[sonar_issues_names]
sonar_measures = sonar_measures[sonar_measures_names]

In [144]:
# Select columns of interest
dtypes = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'object']

## Global functions

In [145]:
def delete_na(name_tables, dataframes, dtypes):
    '''
    Objective:
        - Delete all NA's from the dataframe passed
    Input:
        - Name_tables : String of the names of the tables
        - Dataframe : String of the tables and their selected columns
        - Numerical : Numerical types
        
    Output: 
        - Table with the dataset sizes before and after deleting all NA:
        ["Table name", "Total number of rows", "Number of NA", "Total number of rows after deleting NA"]
    '''
    table = PrettyTable()
    table.field_names = ["Table name", "Total number of rows", "Number of NA", "Total number of rows after deleting NA"]
    for i in range(len(name_tables)):
        dataframe_numerical = dataframes[i].select_dtypes(include=dtypes)
        total_rows = dataframe_numerical.shape[0]
        # Rows with na's
        rows_with_NA = sum([True for idx,row in dataframe_numerical.iterrows() if any(row.isnull())])
        # Delete rows that contain na's
        dataframe_numerical = dataframe_numerical.dropna()
        total_rows_without_NA = dataframe_numerical.shape[0]
        table.add_row([name_tables[i] , total_rows, rows_with_NA,total_rows_without_NA])
        dataframes[i] = dataframe_numerical
    return dataframes


In [146]:
def analyse_categorical_variables(table_names, variable_names, dataframes):
    '''
    Objective:
        - Analyse the categorical variables to be encoded
    Input:
        - Table_names : String of the names of the tables
        - Variable_names : String of the categorical variables corresponding to the table
        - Dataframes : String of the tables and their selected columns
        
    Output: 
        - Table with the categorical variables levels
        ["Table name", "Variable name", "Number of levels", "Types"]
    '''
    table = PrettyTable()
    table.field_names = ["Table name", "Variable name", "Number of levels", "Types"]
    for i in range(len(table_names)):
        for j in range(len(variable_names)):
            table.add_row([table_names[i], variable_names[j], len(dataframes[i][variable_names[j]].unique()), dataframes[i][variable_names[j]].unique()])
    print(table)    

In [147]:
def one_hot_encoding(table, variable):
    variable_dummies = pd.get_dummies(table[variable])
    table = table.drop(variable, axis=1)
    table = table.join(variable_dummies)
    return table
    

### NA values
Deleting all NA values from the tables by using the global function implemented above delete_na().

In [148]:
table_names = ["SONAR_MEASURES", "SONAR_ISSUES", "SONAR_ANALYSIS", "JIRA_ISSUES"]
tables = [sonar_measures, sonar_issues, sonar_analysis, jira_issues]
[sonar_measures, sonar_issues, sonar_analysis, jira_issues] = delete_na(table_names, tables, dtypes)

### Categorical Values
The next step is to analyse the categorical variables and encoding them.
For the SONAR_MEASURES, JIRA_ISSUES, SONAR_ANALYSIS table (add more if necessary) there are not categorical varibles.

In [149]:
table_names = ["SONAR_ISSUES"]
variable_names = ["SEVERITY", "STATUS", "EFFORT"]
dataframes = [sonar_issues]
analyse_categorical_variables(table_names, variable_names, dataframes)


+--------------+---------------+------------------+--------------------------------------------------------------------+
|  Table name  | Variable name | Number of levels |                               Types                                |
+--------------+---------------+------------------+--------------------------------------------------------------------+
| SONAR_ISSUES |    SEVERITY   |        5         |           ['INFO' 'MINOR' 'MAJOR' 'CRITICAL' 'BLOCKER']            |
| SONAR_ISSUES |     STATUS    |        1         |                             ['CLOSED']                             |
| SONAR_ISSUES |     EFFORT    |       181        | [1.0000e+01 2.0000e+00 9.0000e+01 1.0000e+00 5.0000e+00 8.0000e+00 |
|              |               |                  |  1.2000e+01 1.4000e+01 3.0000e+01 2.0000e+01 6.0000e+01 2.5000e+01 |
|              |               |                  |  1.1000e+01 1.9000e+01 2.8000e+01 5.6000e+01 6.0000e+00 1.3000e+01 |
|              |               |

As can be seen in the chunk above, the SEVERITY and STATUS variables have 5 and 1 levels respectively. In our case, we have performed the One-hot encoding for the SEVERITY variable. For the STATUS variable, efore deleting all NA, there was the OPENED level. However, all rows with an OPENED status contained NA, which means that for this variable we only have the CLOSED level. 
When joining the tables, we will calulate the mean of each types each author has.

In [150]:
sonar_issues = one_hot_encoding(sonar_issues, "SEVERITY")

In [151]:
sonar_issues = one_hot_encoding(sonar_issues, "STATUS")
sonar_issues.head()

,CREATION_ANALYSIS_KEY,EFFORT,MESSAGE,START_LINE,END_LINE,CLOSE_ANALYSIS_KEY,BLOCKER,CRITICAL,INFO,MAJOR,MINOR,CLOSED
41,AWd5_psxC4KKKThc-qK6,10.0,Do not forget to remove this deprecated code s...,104.0,104.0,AWeDrnWEC4KKKThcAtWf,0,0,1,0,0,1
198,AWd5_psxC4KKKThc-qK6,10.0,Do not forget to remove this deprecated code s...,354.0,354.0,AWeMr1K3C4KKKThcB9hi,0,0,1,0,0,1
199,AWd5_psxC4KKKThc-qK6,10.0,Do not forget to remove this deprecated code s...,600.0,600.0,AWeMr1K3C4KKKThcB9hi,0,0,1,0,0,1
200,AWd5_psxC4KKKThc-qK6,10.0,Do not forget to remove this deprecated code s...,609.0,609.0,AWeMr1K3C4KKKThcB9hi,0,0,1,0,0,1
201,AWd5_psxC4KKKThc-qK6,10.0,Do not forget to remove this deprecated code s...,619.0,619.0,AWeMr1K3C4KKKThcB9hi,0,0,1,0,0,1


### MESSAGE variable
In the following section, we will encode the MESSAGE variable for the SONAR_ISSUES table. For that variable, we will calulate the length fo the message for each issue, and reassigning the column with that new value instead of the initial message.

In [160]:
message_length = []
for row in sonar_issues['MESSAGE']:
    message_length.append(len(row))
message_length
len(message_length)

728409

In [161]:
# reassign the MESSAGE variable to its length instead of the initial string 
sonar_issues['MESSAGE'] = message_length
sonar_issues.head()

,CREATION_ANALYSIS_KEY,EFFORT,MESSAGE,START_LINE,END_LINE,CLOSE_ANALYSIS_KEY,BLOCKER,CRITICAL,INFO,MAJOR,MINOR,CLOSED
41,AWd5_psxC4KKKThc-qK6,10.0,53,104.0,104.0,AWeDrnWEC4KKKThcAtWf,0,0,1,0,0,1
198,AWd5_psxC4KKKThc-qK6,10.0,53,354.0,354.0,AWeMr1K3C4KKKThcB9hi,0,0,1,0,0,1
199,AWd5_psxC4KKKThc-qK6,10.0,53,600.0,600.0,AWeMr1K3C4KKKThcB9hi,0,0,1,0,0,1
200,AWd5_psxC4KKKThc-qK6,10.0,53,609.0,609.0,AWeMr1K3C4KKKThcB9hi,0,0,1,0,0,1
201,AWd5_psxC4KKKThc-qK6,10.0,53,619.0,619.0,AWeMr1K3C4KKKThcB9hi,0,0,1,0,0,1
